In [1]:
import pickle
import itertools

In [2]:
#G = pickle.load(open('../C-graph1/OUTPUT/UV/column-a-graph.pkl', "rb"))['G']

In [3]:
C = pickle.load(open('../C-graph1/OUTPUT/UV/column-b-cliques.pkl', "rb"))['C']

In [6]:
# Each clique as sorted sequence
C = [tuple(sorted(c)) for c in C]

In [7]:
"Number of maximal cliques: {} million".format(round(len(C) / 1e6))

'Number of maximal cliques: 41 million'

In [10]:
n = 5
nc2maxc = dict([])
# i = maximal clique index
for i in range(0, len(C)) :
    # List of n-cliques belonging to max clique C[i]
    nC = list(itertools.combinations(C[i], n))
    if (len(nC) == 0) : continue
    for nc in nC :
        nc2maxc[nc] = i

len(nc2maxc)

7049975

In [55]:
list(itertools.combinations(tuple([1, 2, 3]), 2))

[(1, 2), (1, 3), (2, 3)]